In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from io import StringIO

# Load pre-trained model and vectorizer
@st.cache_resource
def load_model():
    with open('model.pkl', 'rb') as model_file:
        model = pickle.load(model_file)
    with open('vectorizer.pkl', 'rb') as vectorizer_file:
        vectorizer = pickle.load(vectorizer_file)
    return model, vectorizer

model, vectorizer = load_model()

# App title and description
st.title("Hate Speech Detection App")
st.write("This app uses a machine learning model to detect hate speech in text. Provide input as a single sentence or upload a file containing multiple sentences.")

# Single text input
st.header("Analyze a Single Sentence")
input_text = st.text_area("Enter a sentence to analyze:", "")
if st.button("Analyze Sentence"):
    if input_text.strip():
        input_vectorized = vectorizer.transform([input_text])
        prediction = model.predict(input_vectorized)[0]
        confidence = np.max(model.predict_proba(input_vectorized)) * 100
        st.write(f"Prediction: **{'Hate Speech' if prediction == 1 else 'Not Hate Speech'}**")
        st.write(f"Confidence: **{confidence:.2f}%**")
    else:
        st.error("Please enter a valid sentence.")

# File upload for batch analysis
st.header("Analyze a File")
uploaded_file = st.file_uploader("Upload a .txt or .csv file:", type=["txt", "csv"])
if uploaded_file:
    try:
        if uploaded_file.name.endswith(".txt"):
            # Read .txt file
            content = StringIO(uploaded_file.getvalue().decode("utf-8")).readlines()
            data = pd.DataFrame(content, columns=["text"])
        elif uploaded_file.name.endswith(".csv"):
            # Read .csv file
            data = pd.read_csv(uploaded_file)
            if "text" not in data.columns:
                st.error("The CSV file must contain a 'text' column.")
                st.stop()

        # Analyze each line in the file
        st.write("Analyzing uploaded file...")
        data['prediction'] = model.predict(vectorizer.transform(data['text']))
        data['confidence'] = model.predict_proba(vectorizer.transform(data['text'])).max(axis=1) * 100
        data['prediction_label'] = data['prediction'].apply(lambda x: 'Hate Speech' if x == 1 else 'Not Hate Speech')

        st.write("Analysis complete!")
        st.dataframe(data[['text', 'prediction_label', 'confidence']])

        # Downloadable results
        csv = data.to_csv(index=False).encode('utf-8')
        st.download_button("Download Results as CSV", data=csv, file_name="predictions.csv", mime="text/csv")
    except Exception as e:
        st.error(f"An error occurred while processing the file: {e}")

# Optional: Explainability using SHAP
st.header("Model Explainability (Optional)")
st.write("Feature contributions and explainability coming soon!")

# Footer
st.write("---")
st.write("Developed by [Your Name]")


In [ ]:
# Add this at the beginning of your code
st.set_page_config(
    page_title="Hate Speech Detection",
    page_icon="🔍",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .main {
        padding: 2rem;
        border-radius: 0.5rem;
    }
    .stTitle {
        color: #2c3e50;
        font-size: 3rem !important;
        padding-bottom: 2rem;
        text-align: center;
    }
    .stHeader {
        color: #34495e;
        padding-top: 2rem;
    }
    .stTextArea {
        border: 2px solid #3498db;
        border-radius: 0.5rem;
    }
    .stButton>button {
        background-color: #3498db;
        color: white;
        padding: 0.5rem 2rem;
        border-radius: 0.3rem;
        border: none;
        transition: all 0.3s;
    }
    .stButton>button:hover {
        background-color: #2980b9;
        transform: translateY(-2px);
    }
    .stDownloadButton>button {
        background-color: #27ae60;
        color: white;
    }
    .stDownloadButton>button:hover {
        background-color: #219a52;
    }
    .css-1d391kg {
        padding: 2rem;
        background-color: #f8f9fa;
        border-radius: 0.5rem;
    }
    .stDataFrame {
        border: 1px solid #e0e0e0;
        border-radius: 0.5rem;
        padding: 1rem;
    }
    footer {
        text-align: center;
        padding: 2rem;
        background-color: #f8f9fa;
        margin-top: 3rem;
    }
    .prediction-box {
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 1rem 0;
        text-align: center;
    }
    .hate-speech {
        background-color: #ff7675;
        color: white;
    }
    .not-hate-speech {
        background-color: #00b894;
        color: white;
    }
</style>
""", unsafe_allow_html=True)

# Your existing code here...

# Modify the prediction display
if st.button("Analyze Sentence"):
    if input_text.strip():
        input_vectorized = vectorizer.transform([input_text])
        prediction = model.predict(input_vectorized)[0]
        confidence = np.max(model.predict_proba(input_vectorized)) * 100
        
        prediction_class = 'hate-speech' if prediction == 1 else 'not-hate-speech'
        st.markdown(f"""
            <div class="prediction-box {prediction_class}">
                <h3>Prediction: {'Hate Speech' if prediction == 1 else 'Not Hate Speech'}</h3>
                <p>Confidence: {confidence:.2f}%</p>
            </div>
        """, unsafe_allow_html=True)
    else:
        st.error("Please enter a valid sentence.")

# Modified footer
st.markdown("""
<footer>
    <p>Developed with ❤️ by [Your Name]</p>
</footer>
""", unsafe_allow_html=True)
